In [1]:
import numpy as np
import import_ipynb
import time
from numpy import genfromtxt
import rtree
from rtree import index

In [8]:
batch_size = 100000
TOTAL_SIZE = 100000
for i in range(10):
    s_time = time.time()
    dataset = genfromtxt('C:/Users/Cloud/iCloudDrive/HUAWEI_LKD/Dataset/Legacy/data/TPCH_12M_8Field.csv', delimiter=',',
                        skip_header = i*batch_size, max_rows = batch_size)
    e_time = time.time()
    print('- - -',i,'- - - ')
    print('load time: ', e_time - s_time)

- - - 0 - - - 
load time:  0.7495675086975098
- - - 1 - - - 
load time:  0.8139548301696777
- - - 2 - - - 
load time:  0.855985164642334
- - - 3 - - - 
load time:  0.9320805072784424
- - - 4 - - - 
load time:  1.012078046798706
- - - 5 - - - 
load time:  1.1780333518981934
- - - 6 - - - 
load time:  1.167330026626587
- - - 7 - - - 
load time:  1.2551357746124268
- - - 8 - - - 
load time:  1.4044301509857178
- - - 9 - - - 
load time:  1.4435057640075684


In [ ]:
# conclusion: numpy method is not good

In [9]:
import pandas as pd

In [15]:
last_time = time.time()
for chunk_df in pd.read_csv('C:/Users/Cloud/iCloudDrive/HUAWEI_LKD/Dataset/Legacy/data/TPCH_12M_8Field.csv', chunksize=1000000):
    current_time = time.time()
    print('process time: ',current_time - last_time)
    print(len(chunk_df))
    last_time = current_time

process time:  1.2424235343933105
1000000
process time:  1.235649585723877
1000000
process time:  1.2551043033599854
1000000
process time:  1.2444002628326416
1000000
process time:  1.1985268592834473
1000000
process time:  1.1965761184692383
1000000
process time:  1.2083215713500977
1000000
process time:  1.1897449493408203
1000000
process time:  1.1985223293304443
1000000
process time:  1.1819350719451904
1000000
process time:  1.2297420501708984
1000000
process time:  1.2746565341949463
997995


In [ ]:
# conclusion: pandas method for chunk processing is acceptable

In [4]:
from Utils import *

In [208]:
# begin_time = time.time()

# partition_path = 'D:/iCloudDrive/HUAWEI_LKD/HDFSExperiment/generated_partitions/nora_partitions'
# partitions = load_partitions_from_file(partition_path)

# p = index.Property()
# p.leaf_capacity = 32 # cannot be less than 100, indicate the maximum capacity
# p.index_capacity = 32
# p.NearMinimumOverlapFactor = 16
# p.fill_factor = 0.8
# p.overwrite = True

# pd_dict={}

# # create index for the kdnodes
# pidx = index.Index(properties = p) # Rtree index
# for i in range(len(partitions)):
#     #pidx.insert(partitions[i][-4], kdnode_2_border(partitions[i]))
#     pidx.insert(i, kdnode_2_border(partitions[i]))
    
# end_time = time.time()   
# print('total process time: ',end_time - begin_time)

total process time:  0.1064152717590332


In [193]:
# try batch read tbl format
import pandas as pd

def kdnode_2_border(kdnode):
    lower = [domain[0] for domain in kdnode[0]]
    upper = [domain[1] for domain in kdnode[0]]
    border = tuple(lower + upper) # non interleave
    return border

table_path = 'D:/iCloudDrive/HUAWEI_LKD/9a84f6cd-727f-4f10-ae95-10a0214e10a4-tpc-h-tool/2.18.0_rc2/dbgen/lineitem.tbl'

col_names = ['_c'+str(i) for i in range(16)] # using the last column as pid
cols = [i for i in range(16)]

partition_path = 'D:/iCloudDrive/HUAWEI_LKD/HDFSExperiment/generated_partitions/qd_tree_partitions'
# partition_path = '~/PartitionLayout/qd_tree_partitions'
partitions = load_partitions_from_file(partition_path)

p = index.Property()
p.leaf_capacity = 32 # cannot be less than 100, indicate the maximum capacity
p.index_capacity = 32
p.NearMinimumOverlapFactor = 16
p.fill_factor = 0.8
p.overwrite = True

pd_dict={}

# create index for the kdnodes
pidx = index.Index(properties = p) # Rtree index
for i in range(len(partitions)):
    #pidx.insert(partitions[i][-4], kdnode_2_border(partitions[i]))
    pidx.insert(i, kdnode_2_border(partitions[i]))
    
def find_pid(row, used_dims):
    row_used_dims = row[used_dims]
    #print('row_used_dims: ',row_used_dims)
    row_list = row_used_dims.values.tolist()
    #print('row_list: ', row_list)
    border = tuple(row_list+row_list)
    #print('border: ',border)
    overlap_pid = list(pidx.intersection(border))
    return overlap_pid[0]

def find_pid2(row):
    #print('row: ', row)
    row_list = row.values.tolist()
    border = tuple(row_list+row_list)
    #print('border: ',border)
    return list(pidx.intersection(border))[0]

def find_pid3(row, used_dims):
    row_used_dims = row[used_dims]
    row_list = row_used_dims.values.tolist()
    border = tuple(row_list+row_list)
    pid = list(pidx.intersection(border))[0]
    if pid in pd_dict:
        pd_dict[pid].loc[len(pd_dict[pid])] = row.values.tolist() #pd_dict[pid].append(row, ignore_index=True)
    else:
        pd_dict.update({pid: pd.DataFrame(columns=col_names)})
        pd_dict[pid].loc[0] = row.values.tolist()

def find_pid4(row, used_dims):
    row_used_dims = row[used_dims]
    row_list = row_used_dims.values.tolist()
    border = tuple(row_list+row_list)
    pid = list(pidx.intersection(border))[0]
    if pid in pd_dict:
        pd_dict[pid].append(row.values.tolist())  #pd_dict[pid].append(row, ignore_index=True)
    else:
        pd_dict.update({pid: []})
        pd_dict[pid].append(row.values.tolist())
        
def find_pid5(row, used_dims, pidx):
    row_numpy = row.to_numpy()
    row_list = row_numpy[used_dims].tolist()
    border = tuple(row_list+row_list)
    pid = list(pidx.intersection(border))[0]
    if pid in pd_dict:
        pd_dict[pid].append(row_numpy.tolist())  #pd_dict[pid].append(row, ignore_index=True)
    else:
        pd_dict.update({pid: []})
        pd_dict[pid].append(row_numpy.tolist())
        #print(pd_dict[pid])
        
def find_pid6(row, used_dims):
    row_numpy = row.to_numpy()
    row_list = row_numpy[used_dims].tolist()
    border = tuple(row_list+row_list)
    pid = list(pidx.intersection(border))[0]
    if pid in pd_dict:
        np.vstack((pd_dict[pid], row_numpy))  #pd_dict[pid].append(row, ignore_index=True)
    else:
        pd_dict.update({pid: row_numpy})
        
used_dims = [1,2]

# begin_time = time.time()
# last_time = time.time()

# select_col_names = ['_c1','_c2']

# count = 0
# for chunk in pandas.read_table(table_path, delimiter='|', usecols=cols, names=col_names, chunksize=10):
#     print('chunk id ', count)
#     #print(chunk.iloc[:,used_dims])
#     print(chunk[select_col_names])
#     #chunk['pid'] = chunk.apply(lambda row: find_pid(row, used_dims), axis = 1) 
#     #chunk['pid'] = chunk.iloc[:,1:3].apply(find_pid2) the vectorization operation can only support 1 dimension
#     count += 1
#     current_time = time.time()
#     print('chunk process time: ',current_time - last_time)
#     last_time = current_time
#     if count > 0:
#         break
        
# end_time = time.time()   
# print('total process time: ',end_time - begin_time)

In [194]:
# begin_time = time.time()
# pd_dict = {}
# for i in range(1000):
#     pd_dict.update({i:[]})
# end_time = time.time()   
# print('total process time: ',end_time - begin_time)

In [205]:
pid_data_dict = {}
for k in [9, 12, 16, 0, 36, 32, 1, 24, 20, 41, 28, 17, 44, 4, 52, 8, 48]:
    pid_data_dict.update({k:[]})
    
def process_chunk_row(row, used_dims, pidx):
    row_numpy = row.to_numpy()
    row_used_dims_list = row_numpy[used_dims].tolist()
    row_border = tuple(row_used_dims_list+row_used_dims_list)
    pid = list(pidx.intersection(row_border))[0]
    pid_data_dict[pid].append(row_numpy.tolist())
#     if pid in pd_dict:
#         pd_dict[pid].append(row_numpy.tolist())  #pd_dict[pid].append(row, ignore_index=True)
#     else:
#         pd_dict.update({pid: []})
#         pd_dict[pid].append(row_numpy.tolist())

In [195]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [192]:
#%%timeit
# 1.49 s ± 32.8 ms

begin_time = time.time()

count = 0
for chunk in pandas.read_table(table_path, delimiter='|', usecols=cols, names=col_names, chunksize=100000):
    #print('chunk id ', count)
    # %lprun -f find_pid chunk.apply(lambda row: find_pid(row, used_dims), axis = 1)
    #chunk['pid'] = chunk.apply(lambda row: find_pid(row, used_dims), axis = 1)
    
    #chunk.apply(lambda row: find_pid3(row, used_dims), axis = 1) # totally 44s for 1W rows
    #%lprun -f find_pid3 chunk.apply(lambda row: find_pid3(row, used_dims), axis = 1)
    
    # chunk.apply(lambda row: find_pid4(row, used_dims), axis = 1)
    #%lprun -f find_pid4 chunk.apply(lambda row: find_pid4(row, used_dims), axis = 1) 
    
#     chunk.apply(lambda row: find_pid5(row, used_dims, pidx), axis = 1)
#     chunk.apply(lambda row: find_pid6(row, used_dims), axis = 1)

    chunk.apply(lambda row: process_chunk_row(row, used_dims, pidx), axis = 1)
    
    count += 1
    if count > 0:
        break 
        
end_time = time.time()   
print('total process time: ',end_time - begin_time)

total process time:  3.4892258644104004


In [165]:
import threading

dict_pid = {1:0, 2:0}

class myThread(threading.Thread):
    def __init__(self, tid, name, dict_pid, key, value):
        threading.Thread.__init__(self)
        self.tid = tid
        self.name = name
        self.dict_pid = dict_pid
        self.key = key
        self.value = value
        
    def run(self):
        self.dict_pid[self.key] += self.value

thread1 = myThread(1, "Thread-1", dict_pid, 1, 2)
thread2 = myThread(2, "Thread-2", dict_pid, 2, 2)
thread3 = myThread(3, "Thread-3", dict_pid, 2, 1)
thread4 = myThread(4, "Thread-4", dict_pid, 1, 3)


# 开启新线程
thread1.start()
thread2.start()
thread3.start()
thread4.start()

In [166]:
print(dict_pid)

{1: 5, 2: 3}


In [5]:
from mem_top import mem_top
print(mem_top())
test_arr = []
for i in range(10000000):
    test_arr.append(i)
print(mem_top())


refs:
10000000	<class 'list'> [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 2
5696	<class 'list'> ['# module pyparsing.py\n', '#\n', '# Copyright (c) 2003-2016  Paul T. McGuire\n', '#\n', '# Permiss
3225	<class 'list'> ['# -*- coding: utf-8 -*-\n', '"""Main IPython class."""\n', '\n', '#-------------------------------
2576	<class 'dict'> {2003551152: <weakref at 0x00000213B4CD4638; to 'type' at 0x00000000776BC3B0 (type)>, 2003554368: <w
2028	<class 'dict'> {'__name__': 'lib', '__doc__': None, '__package__': None, '__loader__': None, '__spec__': None, '_or
2022	<class 'dict'> {'Cryptography_HAS_CMS': 1, 'Cryptography_HAS_EC2M': 1, 'Cryptography_HAS_EC_1_0_2': 1, 'Cryptograph
1280	<class 'dict'> {'builtins': <module 'builtins' (built-in)>, 'sys': <module 'sys' (built-in)>, '_frozen_importlib': 
704	<class 'dict'> {(<class 'str'>, '(-?(?:0|[1-9]\\d*))(\\.\\d+)?([eE][-+]?\\d+)?', <RegexFlag.VERBOSE|DOTALL|MULTILIN
646	<class 'set'> {'py

In [16]:
threading.enumerate()

[<_MainThread(MainThread, started 134080)>,
 <Thread(Thread-4, started daemon 137280)>,
 <Heartbeat(Thread-5, started daemon 89848)>,
 <HistorySavingThread(IPythonHistorySavingThread, started 138684)>,
 <ParentPollerWindows(Thread-3, started daemon 35224)>]